In [ ]:
import sys
print(sys.executable)
# !{sys.executable} -m pip install -U pip
# !{sys.executable} -m pip install -U python-dotenv

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# 프롬프트 템플릿 활용
from langchain_core.prompts import PromptTemplate

# {} 에 입력 변수
template = "{country}의 수도는 어디인가요?"

prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [ ]:
# from_template -> format
prompt = prompt_template.format(country="대한민국")         # {} 부분만 치환해서 사용하는 템플릿
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-5-nano",
    max_tokens=2048,
    temperature=0.1,
)

In [ ]:
# 프롬프트를 만들어서 LLM에게 넣는게 아니라 프롬프트와 LLM을 묶어서 사용한다 LCEL
# LangChain Expression Language
from langchain_core.output_parsers import StrOutputParser

# 프롬프트
prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")         # {} 쪽에 format으로 넣는다

# LLM 모델
model = ChatOpenAI()

# Output Parser : 출력파서 (원하는 형식으로 답변 형식 지정)
output_parser = StrOutputParser()

# 프롬프트 넣고 모델로 전달하고 출력파서로 전달 (순서대로)
chain = prompt | model | output_parser
chain

PromptTemplate(input_variables=['how', 'topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 {how} 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023DFF33CE60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023DFF33EED0>, root_client=<openai.OpenAI object at 0x0000023DDCE1DDC0>, root_async_client=<openai.AsyncOpenAI object at 0x0000023DFF33CFB0>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [12]:
# {} 부분은 채워넣어야만 한다
dict_data = {"topic":"인공지능 모델의 학습 원리", "how":"짧게"}       # ==> 인공지능 모델의 학습 원리 에 대해 짧게 설명해주세요.
chain.invoke(dict_data)

'인공지능 모델은 데이터를 입력으로 받아들여 학습을 진행합니다. 모델은 입력 데이터로부터 일정한 패턴이나 규칙을 학습하고, 이를 바탕으로 새로운 데이터에 대한 예측이나 분류를 수행합니다. 학습 과정에서 모델은 손실 함수를 최소화하도록 가중치를 조정하고, 학습 데이터에 대해 최적의 예측을 만들어내는 것을 목표로 합니다.'

In [15]:
# 프롬프트 변경 (바꿀 부분만 {} 처리)
tempalte = """당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿 from_template
prompt = PromptTemplate.from_template(tempalte)

# LLM
model = ChatOpenAI(model_name="gpt-5-nano")

# parser
output_parser = StrOutputParser()

In [17]:
# 체인 구성
chain = prompt | model | output_parser

answer = chain.invoke({"question":"저는 식당에 가서 음식을 주문하고 싶어요"})

print(answer)

- 영어 회화:
Waiter: Welcome to our restaurant. Do you have a reservation?
Customer: No, just a table for one, please.
Waiter: Certainly. Right this way. Here are your menus. Would you like a few minutes or are you ready to order?
Customer: I’m ready to order. I’ll have the grilled salmon with rice, please.
Waiter: Would you like any sides or a drink with that?
Customer: Yes, I’ll take a Caesar salad and a bottle of sparkling water.
Waiter: Noted. Any allergies we should know about?
Customer: I’m allergic to peanuts.
Waiter: Thanks for letting us know. We’ll make sure your order is prepared without peanuts.
Waiter: Would you like to add anything else?
Customer: No, that’s all for now.
Waiter: All right. Your grilled salmon will be out shortly. Enjoy your meal.
Customer: Thank you.
Customer: Could I get the check, please?
Waiter: Of course. I’ll bring it right over.

- 한글 해석:
웨이터: 저희 레스토랑에 오신 것을 환영합니다. 예약이 있나요?
고객: 아니요, 한 명 자리 부탁드립니다.
웨이터: 물론이죠. 이쪽으로 와 주세요. 메뉴 드립니다. 몇 분 정도 보시겠어요, 아니면 바로 주문하

In [18]:
answer = chain.invoke({"question":"미국에서 피자 주문"})

print(answer)

- 영어 회화:
  Customer: Hi there. I’d like to place an order for delivery, please.
  Order Taker: Sure. What can I get for you today?
  Customer: I’ll have a large pepperoni pizza with extra cheese, and a side of garlic knots.
  Order Taker: Okay. Any other toppings or drinks? And what crust would you like—thin, hand-tossed, or pan?
  Customer: Hand-tossed crust, please. And I’d also like a 2-liter Mountain Dew.
  Order Taker: Got it. Delivery or pickup?
  Customer: Delivery, please. The address is 123 Maple Street, Apt 4B.
  Order Taker: What's the name on the order and the best contact number?
  Customer: Name: Mina Park. Phone: 555-0123.
  Order Taker: Great. Any delivery instructions?
  Customer: Please ring the doorbell and leave it at the door if no one answers.
  Order Taker: Okay. There’s a small delivery fee; your total will be around $28.97. How would you like to pay—online or cash on delivery?
  Customer: I’ll pay on delivery.
  Order Taker: Perfect. Your approximate arrival ti

In [20]:
# 묶어서 2번 전달
answer = chain.batch([{"question":"ChatGPT"}, {"question":"Instagram"}])

print(answer[0])
print(answer[1])

- 영어 회화:
S: Hi ChatGPT, I’m practicing English. Could you help me with a quick dialogue?
GPT: Of course. What would you like to practice?
S: I’d like to practice asking questions and getting explanations.
GPT: No problem. You can phrase questions politely: "Could you explain this grammar rule?" "What does this word mean?" and so on.
S: Could you explain this grammar rule? Today we’re practicing present perfect.
GPT: The present perfect describes actions that happened at an indefinite time or that started in the past and continue to now. You might say: "I have lived here for five years."
S: I have lived here for five years. Is this correct?
GPT: Yes, that’s correct. Great job. Now try a few more.

- 한글 해석:
S: 안녕 ChatGPT, 저는 영어를 연습하고 있어요. 짧은 대화를 도와줄 수 있나요?
GPT: 물론이죠. 무엇을 연습하고 싶나요?
S: 질문하고 설명을 받는 연습을 하고 싶어요.
GPT: 문제 없어요. 정중하게 질문하는 표현으로 이렇게 말할 수 있습니다: "Could you explain this grammar rule?" "What does this word mean?" 등.
S: 이 문법 규칙을 설명해 주실 수 있나요? 오늘은 현재 완료를 연습하고 있어요.
GPT: 현재 완료는 언제인지 모르는 시점

In [21]:
# astream 비동기 스트림 / ainvoke 비동기 호출
async for token in chain.astream({"question":"YouTube"}):
    # 메시지 내용 출력 줄바꿈 없이 바로 출력하고 버퍼 비움
    print(token, end="", flush=True)

- 영어 회화:
  - Hey everyone, welcome back to my channel.
  - If you're new here, make sure to subscribe and turn on notifications so you don't miss future videos.
  - Today, we're diving into [topic], and I'll show you step by step how to [do something].
  - If you have any questions, drop them in the comments and I'll answer.
  - If you enjoyed this video, please hit the like button and share it with your friends.
  - Let's get started!
  - In the next video, we'll cover [related topic].
  - Here's a quick recap of the main points.
  - Thanks for watching — see you in the next one!

- 한글 해석:
  - 안녕하세요 여러분, 제 채널에 다시 오신 것을 환영합니다.
  - 새로 오신 분들은 구독하고 알림을 켜 두면 앞으로의 영상도 놓치지 않을 수 있습니다.
  - 오늘은 [주제]에 대해 파헤쳐 볼 것이고, [무엇을 하는지] 차근차근 보여드리겠습니다.
  - 질문이 있으면 댓글로 남겨 주세요. 제가 답변해 드리겠습니다.
  - 이 영상이 마음에 드셨다면 좋아요를 눌러 주시고 친구들과 공유해 주세요.
  - 자, 시작해 봅시다!
  - 다음 영상에서는 [관련 주제]를 다룰 거예요.
  - 핵심 포인트를 간단히 요약해 드릴게요.
  - 시청해 주셔서 감사합니다 — 다음 영상에서 뵙겠습니다!

In [24]:
# 비동기 호출 ainvoke
my_process = chain.ainvoke({"question":"NVDA"})             # 준비
answer = await my_process            # 실행

print(answer)

- 영어 회화:
- Teacher: Today we're practicing with NVDA, the screen reader.
- Student: I'm excited but a little nervous.
- Teacher: No problem. Let's start by turning it on.
- Student: Where do I find the NVDA key?
- Teacher: On most keyboards, the Insert key acts as the NVDA modifier; on laptops you might use Caps Lock.
- Student: Okay, I turned it on.
- Teacher: Great. Now try to read this sentence.
- Student: It reads it aloud.
- Teacher: Now move to the next link by pressing Tab.
- Student: I can hear "Search."
- Teacher: Good. If you want to read more, you can ask NVDA to read by paragraph.
- Student: Can I adjust the speaking rate?
- Teacher: Yes, you can change it in the NVDA menu.
- Student: What about punctuation?
- Teacher: We can enable punctuation to be announced.
- Student: If I type, will NVDA read what I type?
- Teacher: Yes, you can have it read as you type.
- Student: Thank you.

- 한글 해석:
- 선생님: 오늘은 화면 읽기 프로그램인 NVDA를 사용해 연습해 볼 거예요.
- 학생: 설렘도 있지만 조금 긴장돼요.
- 선생님: 문제 없어요. 먼저